In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

url = 'https://www.avito.ma/fr/maroc/%C3%A0_vendre?o='
page = requests.get(url, headers=headers)
print(page)

In [2]:
!pip install js2xml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.2 MB/s eta 0:00:00


In [13]:
import requests
from bs4 import BeautifulSoup
from js2xml import parse
import pandas as pd

# Lists to store the scraped data in
addressLocality = []
addressRegion = []
categorys = []
product_id = []
telephones = []
publicationType = []
names = []
prices = []
pages = [str(i) for i in range(2, 10000)]

for page in pages:
    url = "https://www.avito.ma/fr/maroc/%C3%A0_vendre?o=" + page
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(max_retries=20)
    session.mount('https://', adapter)
    session.mount('http://', adapter)
    pg = session.get(url)

    if pg.status_code != 200:
        print(f"Failed to fetch page {page}. Status code: {pg.status_code}")
        continue

    html_soup = BeautifulSoup(pg.text, 'html.parser')

    # Extracting the url of each product :
    items_list = html_soup.find(class_="listing listing-thumbs").findAll("div", class_="item-info ctext1 mls")
    items_urls = [i.find('a', href=True)['href'] for i in items_list]

    for url in items_urls:
        session = requests.Session()
        adapter = requests.adapters.HTTPAdapter(max_retries=20)
        session.mount('https://', adapter)
        session.mount('http://', adapter)
        response = session.get(url)

        data = {}

        if response.status_code != 200:
            print(f"Failed to fetch data from URL: {url}. Status code: {response.status_code}")
            continue

        try:
            html_soup = BeautifulSoup(response.text, "html.parser")
            price = html_soup.find("div", class_="panel-body").span.string
            script = html_soup.find("div", class_="container mbm").find_all("script", {"type": "text/javascript"})[-1]
            price = html_soup.find("div", class_="panel-body").span.string
            js_code = script.string
            js_tree = parse(js_code)

            for assign_node in js_tree.xpath('//assign'):
                key = assign_node.find('left/identifier').get('name')
                value = assign_node.find('right').get('value')
                data[key] = value

            # If the data is not empty, then extract:
            if data:
                prices.append(price)  # the price
                addressLocality.append(data.get("addressLocality", ''))  # Local address
                addressRegion.append(data.get("addressRegion", ''))  # Region address
                categorys.append(data.get("category", ''))  # category
                telephones.append(data.get("telephone", ''))  # telephones
                publicationType.append(data.get("publisherType", ''))  # publication type
                names.append(data.get("name", ''))  # the product name
                product_id.append(data.get("id", ''))  # product_id

        except Exception as e:
            print(f"Error while processing URL: {url}. Exception: {e}")

# Save our data in an excel file:
dataset = pd.DataFrame({
    "Product_name": names,
    "Product_id": product_id,
    "Product_Category": categorys,
    "price": prices,
    "Phone_number": telephones,
    "Professional_Publication": publicationType,
    "Region_address": addressRegion,
    "Local_address": addressLocality
})
dataset.to_csv("MyAvitoDataset.csv")